## PROYECTO OPTIMIZACIÓN DEL TALENTO PARA ABC CORPORATION

## INFORME DE VALORES NULOS

En este archivo incluimos el estudio de los valores nulos por columna y las decisiones tomadas con respecto a ellos para facilitar el análisis de los datos.

In [1]:
# Importamos las librerias a utilizar
import pandas as pd
import numpy as np


# Configuración para poder visualizar todas las columnas del DataFrame
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv("datos_limpios.csv", index_col=0)
df.head(2)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,51,no,NaN,2015.72,NaN,6,3,NaN,1,1,1,male,NaN,3,5,research director,3,NaN,16280.83,42330.17,7,y,no,13,3.0,3,full time,0,NaN,5,3.0,20,NaN,15,15,16280.83,1972,195370.0,NaN,NaN,yes
1,52,no,NaN,2063.39,NaN,1,4,life sciences,1,2,3,male,NaN,2,5,manager,3,NaN,NaN,43331.17,0,y,NaN,14,3.0,1,NaN,1,34.0,5,3.0,33,NaN,11,9,NaN,1971,199990.0,NaN,NaN,yes


A continuación revisamos los valores nulos por columna.

#### COLUMNAS CON VALORES NULOS

In [14]:
df.isnull().sum()[df.isnull().sum() > 0]

businesstravel          801
department             1366
educationfield          774
hourlyrate             1267
maritalstatus           675
monthlyincome           489
over18                  938
overtime                696
performancerating       200
standardhours           351
totalworkingyears       549
worklifebalance         114
yearsincurrentrole     1643
sameasmonthlyincome     489
salary                  285
roledepartament        1366
numberchildren         1678
dtype: int64

#### COLUMNA BUSINESS TRAVEL: Frecuencia de viajes laborales (travel_rarely, travel_frequently, non-travel)

- Índice 2
- 801 valores nulos
- Imputación por moda (travel_rarely)
- No observamos relación con otras columnas (revisamos relación con department, jobrole, distancefromhome, jobinvolvement, joblevel, performancerating, yearsincurrentrole y remotework)

In [4]:
df['businesstravel'].fillna(df['businesstravel'].mode()[0], inplace=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_7508\1056663662.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['businesstravel'].fillna(df['businesstravel'].mode()[0], inplace=True)


#### COLUMNA DEPARTMENT: Departamento en el que trabaja el empleado (research & development, sales, human resources).

- Índice 4
- 1366 valores nulos
- Relacionamos la columna con 'jobrole'. Descubrimos que los valores faltantes pertenecen a 'Manager', por lo que los sustituímos por una nueva categoría llamada 'Executive Team'.

In [5]:
departamentos = {'sales executive': 'sales', 'research scientist': 'research & development', 
                 'laboratory technician': 'research & development', 'manufacturing director': 'research & development', 
                 'healthcare representative': 'research & development', 
                 'sales representative': 'sales', 'research director': 'research & development',
                 'human resources': 'human resources'} # diccionario para rellenar 'deparment' en base a 'jobrole'
def department(jobrole):
    return departamentos.get(jobrole, None) # funcion para agregar los departamentos o ignorar

df['department'] = df['jobrole'].apply(department)
df['department'] = df['department'].fillna('Executive Team')

#### COLUMNA EDUCATION FIELD: Campo de estudio académico del empleado (human resources, life sciences, marketing, medical, technical degree, other).

- Índice 7
- 774 valores nulos
- No tiene relación con otras columnas. Los sustituimos por 'Unknown'.

In [6]:
df['educationfield'] = df['educationfield'].fillna("unknown")

#### COLUMNA HOURLY RATE: Tarifa por hora calculada.

- Índice 12
- 1267 valores nulos
- Relación con la columna dailyrate. 
- Los valores disponibles nos indican que hourlyrate=dailyrate/8, los calculamos:

In [7]:
df['hourlyrate'] = df['hourlyrate'].fillna(df['dailyrate'] / 8)
df["hourlyrate"]=round(df["hourlyrate"], 2)
df["dailyrate"]=round(df["dailyrate"], 2)

#### COLUMNA MARITAL STATUS: Estado civil (married, divorced, single).

- Índice 17
- 675 valores nulos (30%)
- Al no disponer de este dato, lo sustituimos por "Unknown"

In [8]:
df['maritalstatus'] = df['maritalstatus'].fillna("unknown")

#### COLUMNA MONTHLY INCOME: Ingreso mensual estimado en base al salario anual.

- Índice 18
- 489 valores nulos
- Comprobamos que tiene relación con la columna 'Salary': MonthlyIncome = Salary/12
- Rellenamos los valores nulos calculando el ingreso mensual

In [9]:
df['monthlyincome'] = np.where(df['monthlyincome'].isna(), df['salary'] / 12, df['monthlyincome'])
df["monthlyincome"]=round(df["monthlyincome"], 2)
df["salary"]=round(df["salary"], 2)

#### COLUMNA OVER 18: Indica si el empleado tiene más de 18 años (y).

- Índice 21
- 938 valores nulos
- Nulos modificados por 'y', ya que al comparar con la columna "age" comprobamos que todos los empleados son mayores de edad.

In [10]:
df['over18'] = df['over18'].fillna('y')

#### COLUMNA OVERTIME: Indica si el empleado trabaja horas extras (yes, no).

- Índice 22
- 696 valores nulos
- Imputación por moda (valor más frecuente ('No'))

In [11]:
df['overtime'] = df['overtime'].fillna('no')

#### COLUMNA PERFORMANCE RATING: Evaluación de desempeño en una escala numérica (1, 2, 3, 4).

- Índice 24
- 200 valores nulos
- Se trata de una escala ordinal (1-4), pero solo tenemos valores 3.0 y 4.0. 
- Imputación valores nulos por "unknown"

In [12]:
df['performancerating'] = df['performancerating'].fillna("unknown")

#### COLUMNA STANDARD HOURS: Clasificación de jornada (full time, part time).

- Índice 26
- 351 valores nulos
- Imputación valores nulos por "fulltime" (anteriormente fulltime no se registraba)

In [13]:
df['standardhours'].fillna('fulltime', inplace=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_7508\4158376322.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['standardhours'].fillna('fulltime', inplace=True)


#### COLUMNA TOTAL WORKING YEARS: Años totales de experiencia laboral.

- Índice 28
- 549 valores nulos
- Imputación por la media

In [14]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean') 
df['totalworkingyears'] = imputer.fit_transform(df[['totalworkingyears']])
df['totalworkingyears']=df['totalworkingyears'].round(2)

#### COLUMNA WORK LIFE BALANCE: Nivel de balance entre vida personal y laboral (1, 2, 3, 4).

- Índice 30
- 114 valores nulos
- Imputación por moda (valor más frecuente: 3)

In [15]:
df['worklifebalance'].fillna(df['worklifebalance'].mode()[0], inplace=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_7508\828325093.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['worklifebalance'].fillna(df['worklifebalance'].mode()[0], inplace=True)


#### COLUMNA YEARS IN CURRENT ROLE: Años en el rol actual.

- Índice 32
- 1643 valores nulos
- Para esta columna la imputación no posible ya que no disponemos del dato ni podemos relacionarlo con otras columnas.

#### COLUMNA SAME AS MONTHLY INCOME: Ingreso mensual estimado en base al salario anual.

- Índice 35
- 489 valores nulos
- Mismo valor que MonthlyIncome. Tiene relación con 'Salary', podría calcularse diviendo el salario entre 12
- Valoramos la eliminación de esta columna por dar información duplicada

#### COLUMNA SALARY: Salario anual calculado para el empleado.

- Índice 37
- 285 valores nulos
- Lo calculamos a raíz de los datos de Monthly Income (21 días trabajados) 

In [19]:
df['salary'] = df['salary'].fillna(df['monthlyincome'] * 12)

#### COLUMNA ROLE DEPARTMENT: Combinación de rol y departamento.

- Índice 38
- 1366 valores nulos
- Eliminaremos esta columna ya que no aporta información novedosa.

#### COLUMNA NUMBER CHILDREN: Número de hijos del empleado.

- Índice 39
- 1678 valores nulos (Todos los datos de esta columna) 
- Valoramos la eliminación de la columna ya que la información no es relevante.
- En caso de que la empresa desee obtener esta información, deberá ser solicitada

In [20]:
df["numberchildren"].unique()

array([nan])

---------------------

COLUMNAS QUE DESCARTAREMOS DE NUESTRO INFORME:

- Employee Count: No aporta información adicional
- Over 18: A partir de Age la calculamos y comprobamos que todos son Over 18
- Monthly Income: Se puede calcular a raiz de Salary, no aporta información adicional
- Same as Monthly Income: No aporta información adicional
- Hourly Rate: Se puede calcular a raiz de "Daily Rate", no aporta información adicional
- Monthly Rate: Se puede calcular a raiz de "Daily Rate", no aporta información adicional
- Role Departament: Fusiona los datos de "Job Role" y "Department. No aporta información adicional
- Number of Children: Todos los valores son nulos y no podemos calcularlos.

In [22]:
df.columns

Index(['age', 'attrition', 'businesstravel', 'dailyrate', 'department',
       'distancefromhome', 'education', 'educationfield', 'employeecount',
       'employeenumber', 'environmentsatisfaction', 'gender', 'hourlyrate',
       'jobinvolvement', 'joblevel', 'jobrole', 'jobsatisfaction',
       'maritalstatus', 'monthlyincome', 'monthlyrate', 'numcompaniesworked',
       'over18', 'overtime', 'percentsalaryhike', 'performancerating',
       'relationshipsatisfaction', 'standardhours', 'stockoptionlevel',
       'totalworkingyears', 'trainingtimeslastyear', 'worklifebalance',
       'yearsatcompany', 'yearsincurrentrole', 'yearssincelastpromotion',
       'yearswithcurrmanager', 'sameasmonthlyincome', 'datebirth', 'salary',
       'roledepartament', 'numberchildren', 'remotework'],
      dtype='object')

In [24]:
df.drop(columns=['employeecount', 'over18', 'monthlyincome', 'sameasmonthlyincome',
                 'hourlyrate', 'monthlyrate', 'roledepartament', 'numberchildren'], inplace=True)

In [27]:
df.to_csv('CSV_LIMPIADO.csv', index=False)